In [250]:
import pandas as pd
from sqlalchemy import create_engine
import random
import warnings
import zipfile
from pathlib import Path
import pandas as pd
import tensorflow as tf
import tqdm
from dotenv import load_dotenv
import os

load_dotenv()
warnings.filterwarnings("ignore")

In [251]:
POSTGRES_ADDRESS = os.getenv("POSTGRES_ADDRESS")
POSTGRES_PORT = os.getenv("POSTGRES_PORT")
POSTGRES_USERNAME = os.getenv("POSTGRES_USERNAME")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")
POSTGRES_DBNAME = os.getenv("POSTGRES_DBNAME")

postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
                .format(username=POSTGRES_USERNAME,
                        password=POSTGRES_PASSWORD,
                        ipaddress=POSTGRES_ADDRESS,
                        port=POSTGRES_PORT,
                        dbname=POSTGRES_DBNAME))

cnx = create_engine(postgres_str)

data = pd.read_sql_query("SELECT * FROM get_training_data", cnx)
data.fillna("missing", inplace=True)
data.shape

(1934, 6)

In [252]:
data.iloc[random.choices(range(len(data)), k=10)]

,user,item,label,category_1,category_2,category_3
1905,72,73,0,Manga,Giả tưởng,missing
512,30,96,2,Trong nước,Tiểu thuyết,missing
1516,57,108,3,Giả tưởng,Manga,missing
879,41,87,3,Nước ngoài,Tiểu thuyết,missing
818,40,98,3,Trong nước,Tiểu thuyết,missing
1851,71,102,2,Manga,Giả tưởng,missing
1925,74,76,1,Giả tưởng,Manga,missing
1074,44,89,2,Kinh điển,Tiểu thuyết,Nước ngoài
1389,50,74,1,Manga,Giả tưởng,missing
299,24,96,0,Tiểu thuyết,Trong nước,missing


In [253]:
from libreco.data import random_split

train_data, eval_data, test_data = random_split(data, multi_ratios=[0.8, 0.1, 0.1], seed=42)

In [254]:
from libreco.data import DatasetFeat

sparse_col = ["category_1", "category_2", "category_3"]
item_col = ["category_1", "category_2", "category_3"]

train_data, data_info = DatasetFeat.build_trainset(train_data=train_data, user_col=[], item_col=item_col,
                                                   sparse_col=sparse_col, dense_col=[])
eval_data = DatasetFeat.build_evalset(eval_data)
test_data = DatasetFeat.build_testset(test_data)
data_info

n_users: 60, n_items: 52, data density: 49.5513 %

In [255]:
from libreco.algorithms import TwoTower
tf.compat.v1.reset_default_graph()

model = TwoTower(
    task="ranking",
    data_info=data_info,
    embed_size=16,
    n_epochs=30,
    loss_type="cross_entropy",
    lr=0.001,
    batch_size=2048,
    use_bn=True,
    hidden_units=(128, 64, 32),
)

model.fit(
    train_data,
    neg_sampling=True,  # perform negative sampling on training and eval data
    verbose=2,
    shuffle=True,
    eval_data=eval_data,
    metrics=["loss", "roc_auc", "precision", "recall", "ndcg"],
)

model.init_knn(approximate=False, sim_type="cosine")

Training start time: 2024-11-16 22:29:23
total params: 34,368 | embedding params: 2,720 | network params: 31,648


train: 100%|██████████| 2/2 [00:00<00:00,  2.97it/s]


Epoch 1 elapsed: 0.677s
	 train_loss: 2.5348


eval_listwise: 100%|██████████| 1/1 [00:00<00:00, 333.38it/s]


	 eval log_loss: 0.6945
	 eval roc_auc: 0.4565
	 eval precision@10: 0.0096
	 eval recall@10: 0.0272
	 eval ndcg@10: 0.0319


...

	 eval log_loss: 0.6841
	 eval roc_auc: 0.6210
	 eval precision@10: 0.0269
	 eval recall@10: 0.1141
	 eval ndcg@10: 0.1095


train: 100%|██████████| 2/2 [00:00<00:00, 57.14it/s]


Epoch 30 elapsed: 0.041s
	 train_loss: 0.5425


eval_listwise: 100%|██████████| 1/1 [00:00<00:00, 250.63it/s]


	 eval log_loss: 0.6838
	 eval roc_auc: 0.6195
	 eval precision@10: 0.0269
	 eval recall@10: 0.1141
	 eval ndcg@10: 0.1102


In [256]:
from libreco.evaluation import evaluate

evaluate(
    model=model,
    data=test_data,
    neg_sampling=True,  # perform negative sampling on test data
    metrics=["loss", "roc_auc", "precision", "recall", "ndcg"],
)

eval_listwise: 100%|██████████| 1/1 [00:00<00:00, 333.20it/s]


{'loss': 0.6868372136953229,
 'roc_auc': 0.5805080242321181,
 'precision': 0.009615384615384616,
 'recall': 0.04711538461538462,
 'ndcg': 0.0375278449640126}

In [257]:
data_info.save("models", model_name="two_tower")
model.save("models", model_name="two_tower")